<a href="https://colab.research.google.com/github/fegastal/PosTech-DataAnalytics_TechChallenge_3/blob/main/PNAD_Covid19_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install basedosdados

In [3]:
import pandas as pd
import basedosdados as bd ## database management system for Bra# zilian public data
import numpy as np
import seaborn as sns
import os
from matplotlib import pyplot
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, expr, concat, when, udf

spark = SparkSession.builder.appName("PNAD_Covic19").getOrCreate()
spark

In [4]:
# dict_basedosdados = bd.read_table(dataset_id='br_ibge_pnad_covid',
# table_id='dicionario',
# billing_project_id="basedosdados-1")

In [5]:
# base_df = bd.read_table(dataset_id='br_ibge_pnad_covid',
#   table_id='microdados',
#   billing_project_id="basedosdados-1") \
#           .fillna(0)

In [6]:
# base_df_sql = bd.read_sql(query="SELECT  * \
#                                   FROM `basedosdados.br_ibge_pnad_covid.microdados` cov_md \
#                                   WHERE cov_md.mes in (9,10,11)") \
#                                 .fillna(0)


In [7]:
# base_df_sql

In [8]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [9]:
# covid_df = spark.createDataFrame(data = base_df_sql)

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
csv_path = '/content/gdrive/MyDrive/Colab Notebooks/Consolidado_Knime_PNAD_COVID.csv'

In [12]:
df = spark.read.csv(csv_path, inferSchema = True,header=True)

In [17]:
type(df)

pyspark.sql.dataframe.DataFrame

Selecting specific columns from data dictionary

In [18]:
df = df.select(['UF','Ano','V1013','V1022','A002','A003','B0011','B0012','B0013','B0014','B0015','B0018',
                'B0019','B00111','B002','B005','B006','B007','C001','C007D','C007E','C01011','C013','D0051'
           ])

In [19]:
df.show()

+---+----+-----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+----+-----+-----+------+----+-----+
| UF| Ano|V1013|V1022|A002|A003|B0011|B0012|B0013|B0014|B0015|B0018|B0019|B00111|B002|B005|B006|B007|C001|C007D|C007E|C01011|C013|D0051|
+---+----+-----+-----+----+----+-----+-----+-----+-----+-----+-----+-----+------+----+----+----+----+----+-----+-----+------+----+-----+
| 11|2020|    9|    1|  36|   1|    2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|NULL|   1|   1|    6| NULL|     4|NULL|    1|
| 11|2020|    9|    1|  30|   2|    2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|NULL|   2|   1|   20| NULL|     4|NULL|    1|
| 11|2020|    9|    1|  13|   1|    2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|NULL|   1|NULL| NULL| NULL|  NULL|NULL|    1|
| 11|2020|    9|    1|  11|   1|    2|    2|    2|    2|    2|    2|    2|     2|NULL|NULL|NULL|   1|NULL| NULL| NULL|  NULL|NULL|    1|
| 11|2020|    9|    1|  57|   2|    2|   

## Altering data and column names

In [20]:
df

DataFrame[UF: int, Ano: int, V1013: int, V1022: int, A002: int, A003: int, B0011: int, B0012: int, B0013: int, B0014: int, B0015: int, B0018: int, B0019: int, B00111: int, B002: int, B005: int, B006: int, B007: int, C001: int, C007D: int, C007E: int, C01011: int, C013: int, D0051: int]

In [21]:
new_column_name_dict = {
    "UF": 'state_code',
    'Ano': 'year',
    'V1013': 'month',
    "V1022": "housing_status",
    "A002": "age",
    "A003": "sex",
    "B0011": "had_fever_last_week",
    "B0012": "had_cough_last_week",
    "B0013": "had_sore_throat_last_week",
    'B0014': "had_trouble_breathing_last_week",
    "B0015": "had_headache_last_week",
    "B0018": "had_runny_nose_last_week",
    "B0019": "had_lethargy_last_week",
    "B00111": "had_loss_of_smell_last_week",
    "B002": "went_to_healthcare_facility",
    "B005": "was_hospitalized",
    "B006": "used_ventilator",
    "B007": "has_health_insurance",
    "C001": "worked_last_week",
    "C007D": "main_job_activity",
    "C007E": "how_many_employees",
    "C01011": "income_range",
    "C013": "has_worked_remote",
    'D0051': "coronavirus_related_aid"
}

In [22]:
for old_col, new_col in new_column_name_dict.items():
    df = df.withColumnRenamed(old_col, new_col)

In [23]:
df.show()

+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+-----------------+------------------+------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+------------------

In [24]:
symptoms_dict = {
    1: "yes",
    2: "no",
    3: "dont_know",
    9: "ignored",
    0: "not_applicable"
}

In [25]:
# Columns to translate
symptoms_cols = ['had_fever_last_week','had_cough_last_week',
                 'had_sore_throat_last_week','had_headache_last_week','had_runny_nose_last_week',
                 'had_lethargy_last_week','had_loss_of_smell_last_week','went_to_healthcare_facility','was_hospitalized','has_health_insurance',
                 'worked_last_week','has_worked_remote','coronavirus_related_aid','used_ventilator','had_trouble_breathing_last_week']

work_cols = ['main_job_activity','how_many_employees','income_range']

In [26]:
brazil_states = {
    11: "RO",
    12: "AC",
    13: "AM",
    14: "RR",
    15: "PA",
    16: "AP",
    17: "TO",
    21: "MA",
    22: "PI",
    23: "CE",
    24: "RN",
    25: "PB",
    26: "PE",
    27: "AL",
    28: "SE",
    29: "BA",
    31: "MG",
    32: "ES",
    33: "RJ",
    35: "SP",
    41: "PR",
    42: "SC",
    43: "RS",
    50: "MS",
    51: "MT",
    52: "GO",
    53: "DF"
}

In [27]:

sector_translation = {
    1: "agriculture_livestock_forestry_and_fishing",
    2: "extraction_of_petroleum_coal_metallic_minerals_stone_sand_salt_etc",
    3: "manufacturing_industry_including_clothing_and_homemade_manufacturing",
    4: "supply_of_electricity_and_gas_water_sewerage_and_garbage_collection",
    5: "construction",
    6: "wholesale_and_retail_trade",
    7: "repair_of_motor_vehicles_and_motorcycles",
    8: "passenger_transport",
    9: "freight_transport",
    10: "storage_post_and_delivery_services",
    11: "accommodation_hotels_inns_etc",
    12: "food_service_bars_restaurants_food_vendors",
    13: "information_and_communication_newspapers_radio_and_television_telecommunications_and_it",
    14: "banks_financial_and_insurance_activities",
    15: "real_estate_activities",
    16: "law_firms_engineering_advertising_and_veterinary_services_professional_scientific_and_technical_activities",
    17: "labor_leasing_security_cleaning_landscaping_and_call_center_activities",
    18: "public_administration_federal_state_and_municipal_government",
    19: "education",
    20: "human_health_and_social_assistance",
    21: "religious_organizations_unions_and_associations",
    22: "artistic_sports_and_recreational_activities",
    23: "hairdressing_beauty_treatment_and_personal_services",
    24: "paid_domestic_service_to_be_reported_under_occupation",
    25: "other",
    0: "not_applicable"
}

In [28]:
employee_translation = {
    1: "1_to_5",
    2: "6_to_10",
    3: "11+",
    0: "not_applicable"
}

In [29]:
income_range = {
    0: "0-100",
    1: "101-300",
    2: "301-600",
    3: "601-800",
    4: "801-1,600",
    5: "1,601-3,000",
    6: "3,001-10,000",
    7: "10,001-50,000",
    8: "50,001-100,000",
    9: "100,000+",
    0: "not_applicable"
}

In [58]:
df.schema

StructType([StructField('state_code', IntegerType(), True), StructField('year', IntegerType(), True), StructField('month', IntegerType(), True), StructField('housing_status', IntegerType(), True), StructField('age', IntegerType(), True), StructField('sex', IntegerType(), True), StructField('had_fever_last_week', StringType(), True), StructField('had_cough_last_week', StringType(), True), StructField('had_sore_throat_last_week', StringType(), True), StructField('had_trouble_breathing_last_week', StringType(), True), StructField('had_headache_last_week', StringType(), True), StructField('had_runny_nose_last_week', StringType(), True), StructField('had_lethargy_last_week', StringType(), True), StructField('had_loss_of_smell_last_week', StringType(), True), StructField('went_to_healthcare_facility', StringType(), True), StructField('was_hospitalized', StringType(), True), StructField('used_ventilator', StringType(), True), StructField('has_health_insurance', StringType(), True), StructFiel

In [59]:
df.show(5)

+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+-----------------+------------------+------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+------------------

In [40]:
#Viewing the dataframe schema
df.printSchema()

root
 |-- state_code: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- housing_status: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- had_fever_last_week: integer (nullable = true)
 |-- had_cough_last_week: integer (nullable = true)
 |-- had_sore_throat_last_week: integer (nullable = true)
 |-- had_trouble_breathing_last_week: integer (nullable = true)
 |-- had_headache_last_week: integer (nullable = true)
 |-- had_runny_nose_last_week: integer (nullable = true)
 |-- had_lethargy_last_week: integer (nullable = true)
 |-- had_loss_of_smell_last_week: integer (nullable = true)
 |-- went_to_healthcare_facility: integer (nullable = true)
 |-- was_hospitalized: integer (nullable = true)
 |-- used_ventilator: integer (nullable = true)
 |-- has_health_insurance: integer (nullable = true)
 |-- worked_last_week: integer (nullable = true)
 |-- main_job_activity: integer (nullable = tr

In [47]:
def translate_and_cast(value):
    return symptoms_dict.get(value, None)

translate_udf = udf(translate_and_cast, StringType())

# Apply the UDF to the selected columns
for column in symptoms_cols:
    df = df.withColumn(column, translate_udf(col(column)))

In [48]:
df.show()

+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+------------

In [49]:
def translate_and_cast_state(value):
    return brazil_states.get(value, None)

translate_udf = udf(translate_and_cast_state, StringType())

# Apply the UDF to the selected columns
df = df.withColumn("state_code", col("state_code").cast(IntegerType()))
df = df.withColumn('state_code', translate_udf(col('state_code')))

In [50]:
df.show()

+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+------------

In [41]:
def translate_and_cast_sector(value):
    return sector_translation.get(value, None)

translate_udf = udf(translate_and_cast_sector, StringType())

# Apply the UDF to the selected columns

df = df.withColumn('main_job_activity', translate_udf(col('main_job_activity')))

In [42]:
df.show()

+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+------------

In [51]:
def translate_and_cast_n_of_employees(value):
    return employee_translation.get(value, None)

translate_udf = udf(translate_and_cast_sector, StringType())

# Apply the UDF to the selected columns

df = df.withColumn('how_many_employees', translate_udf(col('how_many_employees')))

In [52]:
df.show()

+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+---+-------------------+-------------------+-------------------------+------------

In [53]:
def translate_and_cast_income(value):
    return income_range.get(value, None)

translate_udf = udf(translate_and_cast_income, StringType())

# Apply the UDF to the selected columns

df = df.withColumn('income_range', translate_udf(col('income_range')))

In [54]:
df = df.withColumn("sex", F.expr("CASE WHEN sex == 1 THEN 'male' ELSE 'female' END"))
df = df.withColumn("housing_status", F.expr("CASE WHEN housing_status == 1 THEN 'urban' ELSE 'rural' END"))

In [55]:
df.show()

+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+--------------+-----------------+-----------------------+
|state_code|year|month|housing_status|age|   sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|  income_range|has_worked_remote|coronavirus_related_aid|
+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------

In [ ]:
df.count()

3.1 | Número de sintomas (1 ou mais sintomas, até 5 no máximo)

In [90]:
# creating has_covid column based on the symptoms column

symptom_col = ['had_fever_last_week','had_cough_last_week',
                 'had_sore_throat_last_week','had_headache_last_week','had_runny_nose_last_week',
                 'had_lethargy_last_week','had_loss_of_smell_last_week']

## if a row has a count of symptoms equal or greater than 1, we identify this research as positive for Covid
yes_count = sum(when(col(c) == "yes", 1).otherwise(0) for c in symptom_col)

In [91]:
# Use withColumn to create the has_covid column
df = df.withColumn("has_covid", yes_count >= 1)


In [92]:
probably_covid = df.filter(df.has_covid==True)

In [105]:
not_probably_covid = df.filter(df.has_covid==False)

In [106]:
not_probably_covid.show()

+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+--------------+-----------------+-----------------------+---------+
|state_code|year|month|housing_status|age|   sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|  income_range|has_worked_remote|coronavirus_related_aid|has_covid|
+----------+----+-----+--------------+---+------+-------------------+-------------------+-----

In [93]:
probably_covid.show()

+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+---------+
|state_code|year|month|housing_status|age|   sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|has_covid|
+----------+----+-----+--------------+---+------+-------------------+-------------------+---------

3.2.1 | Teve sintoma -> idade da pessoa? (0->21, 21->60, 60->mais)


In [94]:
probably_covid = probably_covid.withColumn("covid_per_age", when(probably_covid["age"] < 21, "young")
                                  .when((probably_covid["age"] >= 21) & (probably_covid["age"] <= 60), "adult")
                                  .when(probably_covid["age"] > 60, "elderly")
                                  .otherwise("other"))

In [95]:
probably_covid.show()

+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+---------+-------------+
|state_code|year|month|housing_status|age|   sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|has_covid|covid_per_age|
+----------+----+-----+--------------+---+------+-------------------+-

In [96]:
probably_covid.printSchema()

root
 |-- state_code: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- housing_status: string (nullable = false)
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = false)
 |-- had_fever_last_week: string (nullable = true)
 |-- had_cough_last_week: string (nullable = true)
 |-- had_sore_throat_last_week: string (nullable = true)
 |-- had_trouble_breathing_last_week: string (nullable = true)
 |-- had_headache_last_week: string (nullable = true)
 |-- had_runny_nose_last_week: string (nullable = true)
 |-- had_lethargy_last_week: string (nullable = true)
 |-- had_loss_of_smell_last_week: string (nullable = true)
 |-- went_to_healthcare_facility: string (nullable = true)
 |-- was_hospitalized: string (nullable = true)
 |-- used_ventilator: string (nullable = true)
 |-- has_health_insurance: string (nullable = true)
 |-- worked_last_week: string (nullable = true)
 |-- main_job_activity: string (nullable = true)
 |-- how_ma

In [97]:
probably_covid.count()

40511

In [104]:
df.groupby('sex').count().show()

+------+------+
|   sex| count|
+------+------+
|female|598541|
|  male|550656|
+------+------+



3.2.2 | Teve sintoma -> sexo -> homem ou mulher


In [103]:
probably_covid.groupby('sex').count().show()

+------+-----+
|   sex|count|
+------+-----+
|female|23520|
|  male|16991|
+------+-----+



In [107]:
probably_covid.groupby('went_to_healthcare_facility').count().show()

+---------------------------+-----+
|went_to_healthcare_facility|count|
+---------------------------+-----+
|                    ignored|   12|
|                         no|29703|
|                        yes|10796|
+---------------------------+-----+



3.2.3 | Teve sintoma -> foi no médico?


In [110]:
to_healthcare = probably_covid.filter(probably_covid['went_to_healthcare_facility'] == 'yes')

In [111]:
to_healthcare.show()

+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+---------+-------------+
|state_code|year|month|housing_status|age|   sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|has_covid|covid_per_age|
+----------+----+-----+--------------+---+------+-------------------+-

In [115]:
private_health = to_healthcare.filter(to_healthcare['has_health_insurance'] == 'yes')

In [114]:
public_health = to_healthcare.filter(to_healthcare['has_health_insurance'] == 'no')

Teve sintoma -> foi no médico -> SUS ou privado

In [119]:
private_health.count()


3132

In [118]:
public_health.count()

7660

3.2.3.2 | Teve sintoma -> foi no médico -> rentabilidade?


In [121]:
to_healthcare.orderBy('income_range').show()

+----------+----+-----+--------------+---+------+-------------------+-------------------+-------------------------+-------------------------------+----------------------+------------------------+----------------------+---------------------------+---------------------------+----------------+---------------+--------------------+----------------+--------------------+------------------+------------+-----------------+-----------------------+---------+-------------+
|state_code|year|month|housing_status|age|   sex|had_fever_last_week|had_cough_last_week|had_sore_throat_last_week|had_trouble_breathing_last_week|had_headache_last_week|had_runny_nose_last_week|had_lethargy_last_week|had_loss_of_smell_last_week|went_to_healthcare_facility|was_hospitalized|used_ventilator|has_health_insurance|worked_last_week|   main_job_activity|how_many_employees|income_range|has_worked_remote|coronavirus_related_aid|has_covid|covid_per_age|
+----------+----+-----+--------------+---+------+-------------------+-

In [127]:
to_healthcare.groupBy('income_range').count().show()

+--------------+-----+
|  income_range|count|
+--------------+-----+
|          NULL| 5951|
|       601-800|  153|
|     801-1,600| 2142|
|50,001-100,000|    1|
|not_applicable|   29|
|       301-600|  281|
| 10,001-50,000|  130|
|       101-300|  129|
|   1,601-3,000| 1260|
|  3,001-10,000|  720|
+--------------+-----+



3.2.3.3 | Teve sintoma -> foi no médico -> ficou internado?


In [130]:
to_healthcare.groupBy('was_hospitalized').count().show()

+----------------+-----+
|was_hospitalized|count|
+----------------+-----+
|            NULL| 5456|
|       dont_know|   56|
|              no| 4776|
|             yes|  505|
|         ignored|    3|
+----------------+-----+



3.2.4 | Teve sintoma -> ficou em casa? 3.2.4.1 | Teve sintoma -> ficou em casa ou fez office?



In [132]:
df.groupBy('has_worked_remote').count().show()

+-----------------+-------+
|has_worked_remote|  count|
+-----------------+-------+
|             NULL|1100419|
|               no|  14757|
|              yes|  34021|
+-----------------+-------+



In [131]:
to_healthcare.groupBy('has_worked_remote').count().show()

+-----------------+-----+
|has_worked_remote|count|
+-----------------+-----+
|             NULL|10266|
|               no|  127|
|              yes|  403|
+-----------------+-----+



Teve auxílio?



In [133]:
df.groupBy('coronavirus_related_aid').count().show()

+-----------------------+------+
|coronavirus_related_aid| count|
+-----------------------+------+
|                     no|574880|
|                    yes|574317|
+-----------------------+------+



3.2.5 | Teve sintoma -> qual a atividade de trabalho da pessoa?


In [134]:
to_healthcare.groupBy('main_job_activity').count().show()

+--------------------+-----+
|   main_job_activity|count|
+--------------------+-----+
|real_estate_activ...|   27|
|banks_financial_a...|   84|
|repair_of_motor_v...|   71|
|law_firms_enginee...|  168|
|           education|  408|
|food_service_bars...|  226|
|                NULL| 5934|
|accommodation_hot...|   23|
|supply_of_electri...|   47|
|human_health_and_...|  546|
|               other|  293|
|artistic_sports_a...|   42|
|information_and_c...|   98|
| passenger_transport|   75|
|extraction_of_pet...|   22|
|religious_organiz...|   26|
|paid_domestic_ser...|  305|
|wholesale_and_ret...|  686|
|        construction|  221|
|agriculture_lives...|  323|
+--------------------+-----+
only showing top 20 rows



3.3 | Mapa do brasil -> Relacionar estados (uf) com o número de sintomas (2 ou mais sintomas, até 5 no máximo)


## Covid cases per state

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
filtered_df = df_pd_state_covid.query('has_covid==True')
filtered_df

In [ ]:
# setting plot size
plt.figure(figsize=(10, 6))

# creating sns barchart
ax = sns.barplot(x=filtered_df['state_code'].value_counts().index,
                 y=filtered_df['state_code'].value_counts().values,
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('COVID-19 Cases by State, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                size=10,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# setting plot size
plt.figure(figsize=(15, 6))

# creating sns barchart
ax = sns.countplot(data=filtered_df.query('was_hospitalized == "yes" | was_hospitalized=="no"'),
                 x= 'state_code',
                 hue='was_hospitalized',
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('Hospitalized COVID-19 Cases by State, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                size=10,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show

In [ ]:
# setting plot size
plt.figure(figsize=(15, 6))

# creating sns barchart
ax = sns.countplot(data=filtered_df.query('was_hospitalized== "yes" & (used_ventilator == "yes" | used_ventilator=="no")'),
                 x= 'state_code',
                 hue='used_ventilator',
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('Subject used ventilator Cases by State, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center',
                size=10,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show

In [ ]:
# @title
# setting plot size
plt.figure(figsize=(15, 6))

# creating sns barchart
ax = sns.countplot(data=filtered_df.query('went_to_healthcare_facility == "yes" & has_health_insurance != "not_applicable"'),
                 x= 'state_code',
                 hue='has_health_insurance',
                 orient = 'h',
                 palette='rocket')

# Title and label
plt.xlabel('State Code')
plt.ylabel('Count')
plt.title('Subject sought healthcare facility, has_health_insurance breakdown, PNAD Contínua Sep/20 to Nov/20')

#adding data label
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.0f'),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center', va = 'center', rotation = 45,
                size=8,
                xytext = (0, 9),
                textcoords = 'offset points')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show

## Connecting PySpark to BigQuery

In [ ]:
from google.cloud import bigquery
import pandas_gbq

In [ ]:
output_path = "/content/gdrive/MyDrive/colab_notebooks/pnad_covid19_treated.csv"

# Use the write method to save the DataFrame as a CSV file
df.write.csv(output_path, header=True, mode="overwrite")

In [ ]:
# Use the Cloud Storage bucket for temporary BigQuery export data used
# by the connector.
bucket = "pnad_covid19"
spark.conf.set('temporaryGcsBucket', bucket)

In [ ]:
# df.write.format('bigquery') \
#   .option('table', 'pnad_covid29.covid19') \
#   .save()